# Deplyment of semi unsupervised learning on ATLAS OpenData
### Testing of reading in data and trying an auto encoder
Remember to pip3 install keras-tuner to tune for the given session

In [22]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras_tuner as kt
seed = tf.random.set_seed(1)
#import ROOT as R

In [23]:
df_atlas = pd.read_hdf("/storage/shared/data/2lep_df_forML_data_fromRDF.hdf5")
df = pd.read_hdf("/storage/shared/data/2lep_df_forML_bkg_signal_fromRDF.hdf5")

#weights = pd.read_csv("weights.csv")

In [24]:
df.info()
df_atlas.info()
#weights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109683372 entries, 0 to 109683371
Data columns (total 39 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   njet20                     int32  
 1   njet60                     int32  
 2   nbjet60                    int32  
 3   nbjet70                    int32  
 4   nbjet77                    int32  
 5   nbjet85                    int32  
 6   isOS                       int32  
 7   isSF                       int32  
 8   mll                        float32
 9   mt2                        float32
 10  met_et                     float32
 11  met_phi                    float32
 12  lep1_flav                  int32  
 13  lep1_pt                    float32
 14  lep1_eta                   float32
 15  lep1_phi                   float32
 16  lep1_E                     float32
 17  lep1_ptcone30              float32
 18  lep1_etcone20              float32
 19  lep1_trackd0pvunbiased     float32
 20

### Adding more columns 
Because the dataset is lacking the "isSignal" and "weight" columns, we have to create them our self

In [25]:
df["category"].unique()

array(['Zjetsincl', 'Zjets', 'Diboson', 'Higgs', 'singleTop', 'topX',
       'RS_G_ZZ', 'SUSYC1N2', 'Wjets', 'SUSYC1C1', 'dmV_Zll', 'ttbar',
       'GG_ttn1', 'ZPrimett', 'Gee', 'SUSYSlepSlep', 'ZPrimeee',
       'ZPrimemumu', 'Wjetsincl', 'Gmumu', 'TT_directTT'], dtype=object)

In [26]:
signals = ['ZPrimemumu', 'SUSYC1C1', 'RS_G_ZZ', 'SUSYSlepSlep', 'SUSYC1N2', 'ZPrimett', 'ZPrimeee', 'dmV_Zll', 'GG_ttn1', 'TT_directTT', 'Gee', 'Gmumu']

In [27]:
issignal = np.where(df["category"].isin(signals)  , 1, 0)

In [28]:
df = df.assign(isSignal=issignal)

In [31]:
df.head(2)

,njet20,njet60,nbjet60,nbjet70,nbjet77,nbjet85,isOS,isSF,mll,mt2,...,lep2_trackd0pvunbiased,lep2_tracksigd0pvunbiased,lep2_isTightID,lep2_z0,channelNumber,costhstar,weight,category,physdescr,isSignal
0,0,0,0,0,0,0,1,1,83241.078125,203008.812500,...,-0.022843,0.767920,1.0,0.000382,361106,0.113907,0.127318,Zjetsincl,Zee,0
1,1,0,0,0,0,0,1,1,91189.015625,115910.992188,...,-0.001654,0.100329,1.0,-0.136329,361106,0.864449,0.114645,Zjetsincl,Zee,0


In [32]:
df_atlas.head(2)


,njet20,njet60,nbjet60,nbjet70,nbjet77,nbjet85,isOS,isSF,mll,mt2,...,lep2_E,lep2_ptcone30,lep2_etcone20,lep2_trackd0pvunbiased,lep2_tracksigd0pvunbiased,lep2_isTightID,lep2_z0,channelNumber,costhstar,weight
0,1,0,0,0,0,0,1,1,12510.837891,57409.433594,...,10032.247070,1196.712769,-92.074127,-0.000793,0.044793,1.0,-0.024559,300800,0.131353,1.0
1,1,0,0,0,0,0,1,1,91430.445312,192062.109375,...,192062.109375,0.000000,-174.402069,0.003242,0.373168,1.0,-0.064178,300800,0.323297,1.0


### Data separation

In [33]:
categories = df["category"]

background_categories = df[df["isSignal"] == 0]["category"].unique()
signal_df = df[df['category'] == 'SUSYC1C1']

background_df = df[df["isSignal"] == 0]

columns_to_drop = ["category", "isSignal", "physdescr"]


signal_df.drop(columns_to_drop, axis=1, inplace=True)
background_df.drop(columns_to_drop, axis=1, inplace=True)


signal_mc = signal_df#.to_numpy()
background_mc = background_df#.to_numpy()

data = df_atlas

/storage/software/easybuild/software/SciPy-bundle/2020.11-fosscuda-2020b/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [34]:
print(np.shape(background_mc))
print(np.shape(signal_mc))
print(np.shape(data))

(106898318, 37)
(135127, 37)
(11442336, 37)


### Data handling and preperations
Before we train on the data, we need to scale it and split it into a validation and 

In [35]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [36]:
# Split background
X_b_train, X_b_val = train_test_split(background_mc, test_size=0.2, random_state=seed)
# Split signal
#X_s_train, X_s_test = train_test_split(signal_mc, test_size=0.2, random_state=seed)

Now, combine samples for testing

In [37]:
b_train_weights = X_b_train["weight"]
b_val_weights = X_b_val["weight"]
s_test_weights = signal_mc["weight"]

data_weights = data["weight"]

X_b_train.pop("weight")
X_b_val.pop("weight")
signal_mc.pop("weight")
data.pop("weight")


X_s_test = signal_mc


X_test = np.concatenate((X_b_val,X_s_test),0)

y_b_val = np.zeros(X_b_val.shape[0])                                                                                                                                                                                                                                                   
y_s_test = np.ones(X_s_test.shape[0])      
y_test = np.concatenate((y_b_val,y_s_test),0)

In [41]:
print(np.shape(X_b_train))
print(np.shape(signal_mc))
print(np.shape(data))

(85518654, 36)
(135127, 36)
(11442336, 36)


In [43]:
scaler_ae = StandardScaler()#MinMaxScaler()
X_b_train = scaler_ae.fit_transform(X_b_train)                                                                                                                                                                                                                        
X_b_val= scaler_ae.transform(X_b_val)                                                                                                                                                                                                                                 
X_s_test = scaler_ae.transform(X_s_test)
data = scaler_ae.transform(data)

In [44]:
data_shape = np.shape(X_b_train)[1]
number_of_rows = np.shape(X_b_train)[0]
n_vali = np.shape(X_b_val)[0]

random_indices_b = np.random.choice(number_of_rows, size=int(1e6), replace=False)
test_indices_b = np.random.choice(n_vali, size=int(200000), replace=False)

smaller_data = X_b_train[random_indices_b, :]
small_vali = X_b_val[test_indices_b, :]


test_indices_sb = np.random.choice(np.shape(X_test)[0], size=int(200000), replace=False)
X_small_test = X_test[test_indices_sb, :]

in_and_out = X_b_train.shape[0]

### Training
Now we can train on the data

In [45]:

def gridautoencoder(X_b, X_back_test):
    tuner = kt.Hyperband(
        AE_model_builder,
        objective=kt.Objective("val_mse", direction="min"),
        max_epochs=50,
        factor=3,
        directory="GridSearches",
        project_name="AE",
        overwrite=True,
    )

    tuner.search(X_b, X_b, epochs=50, batch_size=512,
                 validation_data=(X_back_test, X_back_test))
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print(
        f"""
    For Encoder: \n 
    First layer has {best_hps.get('num_of_neurons0')} with activation {best_hps.get('0_act')} \n
    Second layer has {best_hps.get('num_of_neurons1')} with activation {best_hps.get('1_act')} \n
    
    Latent layer has {best_hps.get("lat_num")} with activation {best_hps.get('2_act')} \n
    \n
    For Decoder: \n 
    First layer has {best_hps.get('num_of_neurons5')} with activation {best_hps.get('5_act')}\n
    Second layer has {best_hps.get('num_of_neurons6')} with activation {best_hps.get('6_act')}\n
    Third layer has activation {best_hps.get('7_act')}\n
    \n
    with learning rate = {best_hps.get('learning_rate')} and alpha = {best_hps.get('alpha')}
    """
    )

    state = True
    while state == True:
        answ = input("Do you want to save model? (y/n) ")
        if answ == "y":
            name = input("name: ")
            tuner.hypermodel.build(best_hps).save(
                f"../tf_models/model_{name}.h5")
            state = False
            print("Model saved")
        elif answ == "n":
            state = False
            print("Model not saved")


def AE_model_builder(hp):
    

    alpha_choice = hp.Choice("alpha", values=[1., 0.5, 0.1, 0.05, 0.01])
    #get_custom_objects().update({"leakyrelu": tf.keras.layers.LeakyReLU(alpha=alpha_choice)})
    activations = {
        "relu": tf.nn.relu,
        "tanh": tf.nn.tanh,
        "leakyrelu": lambda x: tf.nn.leaky_relu(x, alpha=alpha_choice),
        "linear": tf.keras.activations.linear
    }
    inputs = tf.keras.layers.Input(shape=data_shape, name="encoder_input")
    x = tf.keras.layers.Dense(
        units=hp.Int("num_of_neurons0", min_value=16, max_value=data_shape-1, step=1),
        activation=activations.get(hp.Choice(
            "0_act", ["relu", "tanh", "leakyrelu"])))(inputs)
    x1 = tf.keras.layers.Dense(
        units=hp.Int("num_of_neurons1", min_value=7, max_value=15, step=1),
        activation=activations.get(hp.Choice(
            "1_act", ["relu", "tanh", "leakyrelu","linear"]))
    )(x)
    val = hp.Int("lat_num", min_value=1, max_value=6, step=1)
    x2 = tf.keras.layers.Dense(
        units=val, activation=activations.get(hp.Choice(
            "2_act", ["relu", "tanh", "leakyrelu","linear"]))
    )(x1)
    encoder = tf.keras.Model(inputs, x2, name="encoder")

    latent_input = tf.keras.layers.Input(shape=val, name="decoder_input")
    x = tf.keras.layers.Dense(
        units=hp.Int("num_of_neurons5", min_value=7, max_value=15, step=1),
        activation=activations.get(hp.Choice(
            "5_act", ["relu", "tanh", "leakyrelu","linear"]))
    )(latent_input)
    x1 = tf.keras.layers.Dense(
        units=hp.Int("num_of_neurons6", min_value=16, max_value=data_shape-1, step=1),
        activation=activations.get(hp.Choice(
            "6_act", ["relu", "tanh", "leakyrelu","linear"]))
    )(x)
    output = tf.keras.layers.Dense(
        data_shape, activation=activations.get(hp.Choice(
            "7_act", ["relu", "tanh", "leakyrelu","linear"]))
    )(x1)
    decoder = tf.keras.Model(latent_input, output, name="decoder")

    outputs = decoder(encoder(inputs))
    AE_model = tf.keras.Model(inputs, outputs, name="AE_model")

    hp_learning_rate = hp.Choice("learning_rate", values=[
                                 9e-2, 9.5e-2, 1e-3, 1.5e-3])
    optimizer = tf.keras.optimizers.Adam(hp_learning_rate)
    #optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
    AE_model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])

    return AE_model





In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

with tf.device("/CPU:0"):
    gridautoencoder(smaller_data, small_vali)


In [46]:
#hypermodel = tf.keras.models.load_model("../tf_models/model_prelim_ae_2lep_data.h5")
inputs = tf.keras.layers.Input(shape=data_shape, name="encoder_input")
x = tf.keras.layers.Dense(units=16,activation=tf.keras.layers.LeakyReLU(alpha=0.01))(inputs)
x1 = tf.keras.layers.Dense(units=6,activation="tanh")(x)
val = 6
x2 = tf.keras.layers.Dense(units=val, activation=tf.keras.layers.LeakyReLU(alpha=0.01))(x1)
encoder = tf.keras.Model(inputs, x2, name="encoder")

latent_input = tf.keras.layers.Input(shape=val, name="decoder_input")
x = tf.keras.layers.Dense(units=10,activation="linear")(latent_input)
x1 = tf.keras.layers.Dense(units=14,activation=tf.keras.layers.LeakyReLU(alpha=0.01))(x)
output = tf.keras.layers.Dense(data_shape, activation="linear")(x1)
decoder = tf.keras.Model(latent_input, output, name="decoder")

outputs = decoder(encoder(inputs))
AE_model = tf.keras.Model(inputs, outputs, name="AE_model")

hp_learning_rate = 0.0015
optimizer = tf.keras.optimizers.Adam(hp_learning_rate)
AE_model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])

In [47]:
with tf.device("/CPU:0"):
    AE_model.fit(X_b_train, X_b_train, epochs=2, batch_size=4000, validation_data=(X_b_val, X_b_val), use_multiprocessing=True)

Epoch 1/2
21380/21380 [==============================] - 285s 13ms/step - loss: 0.4549 - mse: 0.4549 - val_loss: 0.4203 - val_mse: 0.4203
Epoch 2/2
21380/21380 [==============================] - 449s 21ms/step - loss: 0.4210 - mse: 0.4210 - val_loss: 0.4093 - val_mse: 0.4093


In [ ]:
#Calculate prediction
#with tf.device("/CPU:0"):
pred_back = AE_model.predict(X_b_val)
print("Background done")


In [ ]:
pred_sig = AE_model.predict(X_s_test)
print("Signal done")
#pred_data = AE_model.predict(X_data)

In [ ]:
pred_data = AE_model.predict(data)
print("ATLAS data done")

### Histograms
Now we implement testing of the data, and stacking of histograms with the reconstruction <br> for the given background processes, a signal, and ATLAS data. <br>
<br>
First for background




In [ ]:
from func import reconstructionError

In [ ]:
recon_err_back = reconstructionError(pred_back, X_b_val)

Then signal

In [ ]:
recon_err_sig = reconstructionError(pred_sig, X_s_test)

An then for actual data

In [ ]:
#recon_data = tf.keras.losses.msle(pred_data, X_data)

In [ ]:
print(np.shape(recon_err_back), np.min(recon_err_back), np.max(recon_err_back))
print(np.shape(recon_err_sig), np.min(recon_err_sig), np.max(recon_err_sig))
#print(np.shape(recon_data), np.min(recon_data), np.max(recon_data))

Then  plot

In [ ]:
b = recon_err_back#/np.max(recon_err_back)
b_s = recon_err_sig#/np.max(recon_err_sig)
#norm_recon_data = recon_data/np.max(recon_data)

histo_data = [b_s,b]#, norm_recon_data])
weight_histo = [s_test_weights,b_val_weights]

In [ ]:
import plot_set
plt.rcParams["figure.figsize"] = (12,9)

fig, ax = plt.subplots()

n_bins= 200
colors = ["green", "red"]
labels= ["Signal", "Background"]
ax.hist(histo_data, n_bins, density=True, stacked=True, histtype='bar', color=colors, label=labels, weights=(s_test_weights, b_val_weights))
ax.legend(prop={'size': 10})
ax.set_title('Reconstruction error histogram with background and signal')
ax.set_xlabel('log10 Reconstruction error')
ax.set_ylabel('#Events')
ax.set_yscale('log')

fig.tight_layout()
plt.savefig("b_s_recon.pdf")
plt.show()

Here we plot the ROC curves for the model

In [ ]:
# ROC
Y_b = np.zeros(X_b_val.shape[0]).reshape(X_b_val.shape[0],1);
Y_s = np.ones(X_s_test.shape[0]).reshape(X_s_test.shape[0],1);
Y_ROC = np.concatenate((Y_s, Y_b),0);

sample_weight = np.concatenate((s_test_weights, b_val_weights),0)

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

rec_errors_ROC = np.concatenate((recon_err_sig,recon_err_back),0)



fpr, tpr, thresholds = roc_curve(Y_ROC, rec_errors_ROC, sample_weight = sample_weight, pos_label=1)
ae_auc = roc_auc_score(Y_ROC, rec_errors_ROC)
figRocAE, axRocAE = plt.subplots()
figRocAE.set_size_inches(12,12)
axRocAE.plot(fpr, tpr, label='ROC curve')
axRocAE.plot([0, 1], [0, 1], 'k--')
axRocAE.set_xlim([0.0, 1.0])
axRocAE.set_ylim([0.0, 1.05])
axRocAE.set_xlabel('False Anomaly Rate')
axRocAE.set_ylabel('True Anomaly Rate')
axRocAE.text(0.4,0.2,"AUC = %.4f" % ae_auc,fontsize=15)
axRocAE.set_title("Autoencoder ROC")

fig.tight_layout()
plt.savefig("b_s_roc_curve.pdf")
plt.show()